In [125]:
from fpylll import *
load('StatBKZ.py')
load('experiments.py')
load('framework/instance_gen.sage')
n = 256
q = 8380417
k = 4
eta = 2
d = 13
g = GF(q)(1753)

In [185]:
def bit_reverse(n):
    return int('{:08b}'.format(n)[::-1], 2)


def gen_M(): # Matrix for NTT2
    M = []
    for i in range(n):
        gg = g**(2*bit_reverse(i) + 1)
        mm = [int(gg**i) for i in range(n)]
        M.append(mm)
    return matrix(GF(q), M)


M = gen_M()
M = block_matrix([[M,0,0,0],[0,M,0,0],[0,0,M,0],[0,0,0,M]])
MI = M.inverse().T

In [209]:
def build_LWE_by_se2(A, M, MI, s, e, b, knowns, sample):

    zeros = n*k - knowns
    zero_index = random.sample(list(range(n*k)), zeros)
    sample_index = random.sample(list(range(n*k)), sample)
    MA = MI*A.T

    v = vector(GF(q), M * s)
    for i in zero_index:
        v[i] = 0

    t1 = v * MI
    t2 = v * MI * A.T - b

    a = []
    es = []
    t2s = []
    for i in zero_index:
        ma = []
        for j in sample_index:
            ma.append(MI[i][j])
        a.append(list(MA[i]) + ma)
        
    for j in sample_index:
        t2s.append(t1[j])
        es.append(-s[j])

    a = Matrix(GF(q), a).echelon_form().change_ring(ZZ)

    b1 = []
    for i in range(zeros):
        b1.append(list(a[i]) + [0])

    for i in range(zeros, n*k+sample):
        b1.append([0] * i + [q] + [0] * (n*k-i+sample))

    b1.append(list(t2) + t2s + [1])
    
    return matrix(ZZ, A), matrix(QQ, b1), concatenate(list(e) + es, [1])


In [211]:
def initialize_from_NTT_instance(knowns, sample, dbdd_class, n, q, D_s, D_e, diag=False, verbosity=2):
    mu_s, s_s = round_to_rational(0.0), round_to_rational(2.0)
    mu_e, s_e = round_to_rational(0.0), round_to_rational(2.0)

    s = [vector([random.randint(-eta, eta) for _ in range(n)]) for _ in range(k)]
    e = [vector([random.randint(-eta, eta) for _ in range(n)]) for _ in range(k)]

    A = []
    for i in range(k):
        a = []
        for j in range(k):
            aa = [[random.randint(0, q-1) for i in range(n)] for _ in range(n)]
            aa = matrix(GF(q), aa)
            a.append(aa)
        A.append(a)

    b = []
    for i in range(k):
        bb = A[i][0] * s[0]
        for j in range(1, k):
            bb += A[i][j] * s[j]
        b.append(bb + e[i])

    A = block_matrix(A)
    s = vector(concatenate(s))
    e = vector(concatenate(e))
    b = vector(concatenate(b))
    
    A, B, u = build_LWE_by_se2(A, M, MI, s, e, b, knowns, sample)
    mu = vec((n*k) * [mu_s] + sample * [mu_e] + [1])
    S = diagonal_matrix((n*k) * [s_s] + sample * [s_e] + [0])
    return A, b, dbdd_class(B, S, mu, u, verbosity=verbosity)


In [219]:
est1 = []
est2 = []
expe = []
trials = 1
do_exp = False
for knowns in range(64, 1024, 64):
  for sample in range(0, 1025, 64):
    b1 = 0
    b2 = 0
    x0 = 0
    wk = 0
    for _ in range(trials):
        A, b, dbdd = initialize_from_NTT_instance(knowns, sample, DBDD_predict_diag, n, q, D_s, D_e)
        #beta1, _ = dbdd.estimate_attack(probabilistic=True)
        beta2, _ = dbdd.estimate_attack(probabilistic=False)
        #b1 += beta1
        b2 += beta2
        if do_exp:
            x, y = dbdd.attack()
            x0 += x
            wk += 1
    #est1.append(b1/trials)
    est2.append(b2/trials)
    if do_exp:
        expe.append(x0/wk)
  print(knowns, est2)

       Attack Estimation      
 ln(dvol)=665.3587704 	 ln(Bvol)=1020.2501269 	 ln(Svol)=709.7827129 	δ(β)=1.000319 
 dim=1025 	 δ=0.000000 	 β=9999.00  
  
       Attack Estimation      
 ln(dvol)=1663.4281875 	 ln(Bvol)=2040.5002538 	 ln(Svol)=754.1441324 	δ(β)=1.001994 
 dim=1089 	 δ=1.001994 	 β=1032.55  
  
       Attack Estimation      
 ln(dvol)=2661.4976046 	 ln(Bvol)=3060.7503807 	 ln(Svol)=798.5055520 	δ(β)=1.002446 
 dim=1153 	 δ=1.002446 	 β=786.93  
  
       Attack Estimation      
 ln(dvol)=3659.5670218 	 ln(Bvol)=4081.0005075 	 ln(Svol)=842.8669716 	δ(β)=1.002816 
 dim=1217 	 δ=1.002817 	 β=650.18  
  
       Attack Estimation      
 ln(dvol)=4657.6364389 	 ln(Bvol)=5101.2506344 	 ln(Svol)=887.2283911 	δ(β)=1.003115 
 dim=1281 	 δ=1.003115 	 β=565.97  
  
       Attack Estimation      
 ln(dvol)=5655.7058560 	 ln(Bvol)=6121.5007613 	 ln(Svol)=931.5898107 	δ(β)=1.003353 
 dim=1345 	 δ=1.003354 	 β=510.63  
  
       Attack Estimation      
 ln(dvol)=6653.7752731 	 ln(Bvol